# Import Library and Dataset

In [1]:
import pandas as pd
import gdown
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error

In [2]:
# Helper Function
def import_from_gdrive(id_file, file_name, sheet_name=None):
    link = 'https://drive.google.com/uc?id=' + id_file
    output_file = '/content/' + file_name
    gdown.download(link, output_file, quiet=False)

    df = pd.read_excel(output_file)
    return df

In [3]:
id_file = '1aPZBBtGwqKiPN9fJpY-LUTCz6mVDuux2'
df = import_from_gdrive(id_file,'final_dataset_converted_rupiah_outlier_handling.xlsx')

Downloading...
From: https://drive.google.com/uc?id=1aPZBBtGwqKiPN9fJpY-LUTCz6mVDuux2
To: /content/final_dataset_converted_rupiah_outlier_handling.xlsx
100%|██████████| 9.83M/9.83M [00:00<00:00, 28.8MB/s]


In [4]:
# df.info()

In [5]:
fix_columns =['Total Household Income',
              'Household Head Sex',
              'Household Head Age',
              'Household Head Marital Status',
              # 'Household Head Highest Grade Completed',
              'Household Head Highest Grade Completed (Simplified)',
              'Total Household Income',
              'Region',
              'Type of Building/House',
              'House Floor Area',
              'Number of bedrooms',
              'Electricity',
              'Tenure Status',
              'Type of Household',
              'Total Number of Family members',
              'Total number of family members employed',
              # 'Number of Car, Jeep, Van',
              # 'Number of Motorcycle/Tricycle',
              # 'Number of Television',
              # 'Number of Airconditioner',
              # 'Number of Personal Computer',
              # 'Number of Cellular phone',
              # 'Number of Refrigerator/Freezer',
              # 'Number of Washing Machine',
              'Number of Kids',
              'Number of Vehicles',
              'Number of Communication Devices',
              'Number of Electronics',
              ]

X =  df.loc[:, fix_columns]
X = X.loc[:,~X.columns.duplicated()]

# Mengubah tipe data int menjadi float
int_columns = X.select_dtypes(include=['int32', 'int64']).columns
X[int_columns] = X[int_columns].astype('float64')

y = df['Total Expenses']

In [6]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21930 entries, 0 to 21929
Data columns (total 18 columns):
 #   Column                                               Non-Null Count  Dtype  
---  ------                                               --------------  -----  
 0   Total Household Income                               21930 non-null  float64
 1   Household Head Sex                                   21930 non-null  object 
 2   Household Head Age                                   21930 non-null  float64
 3   Household Head Marital Status                        21930 non-null  object 
 4   Household Head Highest Grade Completed (Simplified)  21925 non-null  object 
 5   Region                                               21930 non-null  object 
 6   Type of Building/House                               21930 non-null  object 
 7   House Floor Area                                     21930 non-null  float64
 8   Number of bedrooms                                   21930 non-nul

In [7]:
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_features = X.select_dtypes(include=['object', 'category']).columns.tolist()

# Pipeline preprocessing untuk fitur numerik
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())  # Standarisasi
])
# Pipeline preprocessing untuk fitur kategorikal
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder())  # One-hot encoding
])

# Gabungkan pipeline preprocessing untuk fitur numerik dan kategorikal
preprocessor = ColumnTransformer(
    transformers=[
        # ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

X = preprocessor.fit_transform(X)

In [13]:
TEST_SIZE = 0.2
RANDOM_STATE = 42
X_train, X_test, y_train, y_test= train_test_split(X,y, test_size=TEST_SIZE, random_state=RANDOM_STATE)
n_cols = X_train.shape[1]

print("X_train shape : " , X_train.shape)
print("y_train shape : " , y_train.shape)
print("X_test  shape : " , X_test.shape)
print("y_test  shape : " , y_test.shape)

X_train = X_train.toarray()  # Konversi SparseTensor menjadi numpy array
X_test = X_test.toarray()    # Konversi SparseTensor menjadi numpy array

X_train shape :  (17544, 48)
y_train shape :  (17544,)
X_test  shape :  (4386, 48)
y_test  shape :  (4386,)


In [14]:
from keras import backend as K
# Fungsi untuk menghitung RMSE
def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true)))

In [18]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],kernel_initializer='normal')),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(256, activation='relu'),
    # tf.keras.layers.Dense(1, kernel_regularizer=tf.keras.regularizers.l2(0.01))
    tf.keras.layers.Dense(1, kernel_initializer='normal',activation='linear'),
])

# Kompilasi model
# model.compile(optimizer='adam', loss=root_mean_squared_error)
model.compile(optimizer='adam',
              loss=tf.keras.losses.mae,
              metrics=["mae"],)


# Latih model
model.fit(X_train, y_train, epochs=100, batch_size=128
          , verbose=2)

# Evaluasi model
loss = model.evaluate(X_test, y_test, verbose=0)
print('Mean Squared Error on Test Set:', loss)

# Prediksi
predictions = model.predict(X_test)

Epoch 1/100
138/138 - 2s - loss: 111585488.0000 - mae: 111585488.0000 - 2s/epoch - 18ms/step
Epoch 2/100
138/138 - 0s - loss: 59044228.0000 - mae: 59044228.0000 - 398ms/epoch - 3ms/step
Epoch 3/100
138/138 - 0s - loss: 29273986.0000 - mae: 29273986.0000 - 420ms/epoch - 3ms/step
Epoch 4/100
138/138 - 0s - loss: 28420266.0000 - mae: 28420266.0000 - 424ms/epoch - 3ms/step
Epoch 5/100
138/138 - 0s - loss: 27976608.0000 - mae: 27976608.0000 - 387ms/epoch - 3ms/step
Epoch 6/100
138/138 - 0s - loss: 27714330.0000 - mae: 27714330.0000 - 398ms/epoch - 3ms/step
Epoch 7/100
138/138 - 1s - loss: 27560148.0000 - mae: 27560148.0000 - 519ms/epoch - 4ms/step
Epoch 8/100
138/138 - 1s - loss: 27453728.0000 - mae: 27453728.0000 - 570ms/epoch - 4ms/step
Epoch 9/100
138/138 - 1s - loss: 27385718.0000 - mae: 27385718.0000 - 559ms/epoch - 4ms/step
Epoch 10/100
138/138 - 1s - loss: 27331932.0000 - mae: 27331932.0000 - 571ms/epoch - 4ms/step
Epoch 11/100
138/138 - 1s - loss: 27333980.0000 - mae: 27333980.0000 

In [ ]:
#  Mean Squared Error on Test Set: 34,777,004.0 --> Adam, batch_size=128
#  Mean Absolute Error on Test Set : 27,250,516.0 --> Adam, batch_size = 256
#  Mean Absolute Error on Test Set : 27,269,222.0 --> Adam, batch_size = 128

In [ ]:
# Create a model using the Sequential API
model = tf.keras.Sequential([
  tf.keras.layers.Dense(1)
])

model.compile(loss=tf.keras.losses.mae, # mae is short for mean absolute error
              optimizer=tf.keras.optimizers.SGD(), # SGD is short for stochastic gradient descent
              metrics=["mae"])

model.fit(X_train, y_train, epochs=100, batch_size=32, verbose=2)

# Evaluasi model
loss = model.evaluate(X_test, y_test, verbose=0)
print('MAE on Test Set:', loss)

# Prediksi
predictions = model.predict(X_test)




# Result

## With Scalling

In [ ]:
# Epoch 1/100
# 851/851 - 3s - loss: 17508932398350336.0000 - 3s/epoch - 4ms/step
# Epoch 2/100
# 851/851 - 3s - loss: 17474640305717248.0000 - 3s/epoch - 3ms/step
# Epoch 3/100
# 851/851 - 2s - loss: 17372946217566208.0000 - 2s/epoch - 2ms/step
# Epoch 4/100
# 851/851 - 2s - loss: 17186693887033344.0000 - 2s/epoch - 2ms/step
# Epoch 5/100
# 851/851 - 1s - loss: 16908547509977088.0000 - 1s/epoch - 1ms/step
# Epoch 6/100
# 851/851 - 1s - loss: 16534964812120064.0000 - 1s/epoch - 1ms/step
# Epoch 7/100
# 851/851 - 1s - loss: 16062195213271040.0000 - 1s/epoch - 2ms/step
# Epoch 8/100
# 851/851 - 1s - loss: 15487896882511872.0000 - 1s/epoch - 1ms/step
# Epoch 9/100
# 851/851 - 1s - loss: 14817167946022912.0000 - 1s/epoch - 1ms/step
# Epoch 10/100
# 851/851 - 1s - loss: 14054316256002048.0000 - 1s/epoch - 2ms/step
# Epoch 11/100
# 851/851 - 1s - loss: 13203269560041472.0000 - 1s/epoch - 2ms/step
# Epoch 12/100
# 851/851 - 2s - loss: 12275238796525568.0000 - 2s/epoch - 2ms/step
# Epoch 13/100
# 851/851 - 2s - loss: 11286933535719424.0000 - 2s/epoch - 2ms/step
# Epoch 14/100
# 851/851 - 1s - loss: 10252717222002688.0000 - 1s/epoch - 1ms/step
# Epoch 15/100
# 851/851 - 1s - loss: 9198005324349440.0000 - 1s/epoch - 2ms/step
# Epoch 16/100
# 851/851 - 1s - loss: 8149303696556032.0000 - 1s/epoch - 1ms/step
# Epoch 17/100
# 851/851 - 1s - loss: 7135974882541568.0000 - 1s/epoch - 1ms/step
# Epoch 18/100
# 851/851 - 1s - loss: 6177110548807680.0000 - 1s/epoch - 1ms/step
# Epoch 19/100
# 851/851 - 1s - loss: 5305121592311808.0000 - 1s/epoch - 1ms/step
# Epoch 20/100
# 851/851 - 1s - loss: 4552857533546496.0000 - 1s/epoch - 1ms/step
# Epoch 21/100
# 851/851 - 1s - loss: 3935167148195840.0000 - 1s/epoch - 2ms/step
# Epoch 22/100
# 851/851 - 2s - loss: 3464278745022464.0000 - 2s/epoch - 2ms/step
# Epoch 23/100
# 851/851 - 2s - loss: 3140279565549568.0000 - 2s/epoch - 2ms/step
# Epoch 24/100
# 851/851 - 1s - loss: 2934851011346432.0000 - 1s/epoch - 1ms/step
# Epoch 25/100
# 851/851 - 1s - loss: 2808695473831936.0000 - 1s/epoch - 1ms/step
# Epoch 26/100
# 851/851 - 1s - loss: 2727711986417664.0000 - 1s/epoch - 2ms/step
# Epoch 27/100
# 851/851 - 1s - loss: 2665984448004096.0000 - 1s/epoch - 2ms/step
# Epoch 28/100
# 851/851 - 1s - loss: 2613077430239232.0000 - 1s/epoch - 1ms/step
# Epoch 29/100
# 851/851 - 1s - loss: 2566170750222336.0000 - 1s/epoch - 2ms/step
# Epoch 30/100
# 851/851 - 1s - loss: 2523007402639360.0000 - 1s/epoch - 1ms/step
# Epoch 31/100
# 851/851 - 2s - loss: 2482796308201472.0000 - 2s/epoch - 2ms/step
# Epoch 32/100
# 851/851 - 2s - loss: 2444996569464832.0000 - 2s/epoch - 2ms/step
# Epoch 33/100
# 851/851 - 1s - loss: 2409475310878720.0000 - 1s/epoch - 2ms/step
# Epoch 34/100
# 851/851 - 1s - loss: 2376330511384576.0000 - 1s/epoch - 1ms/step
# Epoch 35/100
# 851/851 - 1s - loss: 2345030131908608.0000 - 1s/epoch - 1ms/step
# Epoch 36/100
# 851/851 - 1s - loss: 2315078372163584.0000 - 1s/epoch - 1ms/step
# Epoch 37/100
# 851/851 - 1s - loss: 2286593880621056.0000 - 1s/epoch - 1ms/step
# Epoch 38/100
# 851/851 - 1s - loss: 2259716243718144.0000 - 1s/epoch - 1ms/step
# Epoch 39/100
# 851/851 - 1s - loss: 2234208701382656.0000 - 1s/epoch - 1ms/step
# Epoch 40/100
# 851/851 - 1s - loss: 2210537257566208.0000 - 1s/epoch - 1ms/step
# Epoch 41/100
# 851/851 - 2s - loss: 2189026719170560.0000 - 2s/epoch - 2ms/step
# Epoch 42/100
# 851/851 - 2s - loss: 2167449608781824.0000 - 2s/epoch - 2ms/step
# Epoch 43/100
# 851/851 - 2s - loss: 2146419838287872.0000 - 2s/epoch - 2ms/step
# Epoch 44/100
# 851/851 - 1s - loss: 2126212017160192.0000 - 1s/epoch - 1ms/step
# Epoch 45/100
# 851/851 - 1s - loss: 2106962913263616.0000 - 1s/epoch - 1ms/step
# Epoch 46/100
# 851/851 - 1s - loss: 2088466099732480.0000 - 1s/epoch - 1ms/step
# Epoch 47/100
# 851/851 - 1s - loss: 2070675674103808.0000 - 1s/epoch - 1ms/step
# Epoch 48/100
# 851/851 - 1s - loss: 2053520903634944.0000 - 1s/epoch - 1ms/step
# Epoch 49/100
# 851/851 - 1s - loss: 2037030376701952.0000 - 1s/epoch - 1ms/step
# Epoch 50/100
# 851/851 - 1s - loss: 2021242076921856.0000 - 1s/epoch - 2ms/step
# Epoch 51/100
# 851/851 - 2s - loss: 2005818446708736.0000 - 2s/epoch - 2ms/step
# Epoch 52/100
# 851/851 - 2s - loss: 1991202404564992.0000 - 2s/epoch - 2ms/step
# Epoch 53/100
# 851/851 - 2s - loss: 1976843120934912.0000 - 2s/epoch - 2ms/step
# Epoch 54/100
# 851/851 - 1s - loss: 1962808778424320.0000 - 1s/epoch - 2ms/step
# Epoch 55/100
# 851/851 - 1s - loss: 1949371839021056.0000 - 1s/epoch - 1ms/step
# Epoch 56/100
# 851/851 - 1s - loss: 1936382784176128.0000 - 1s/epoch - 1ms/step
# Epoch 57/100
# 851/851 - 1s - loss: 1923828728987648.0000 - 1s/epoch - 2ms/step
# Epoch 58/100
# 851/851 - 1s - loss: 1911802552123392.0000 - 1s/epoch - 1ms/step
# Epoch 59/100
# 851/851 - 1s - loss: 1900175672999936.0000 - 1s/epoch - 1ms/step
# Epoch 60/100
# 851/851 - 1s - loss: 1888494301478912.0000 - 1s/epoch - 1ms/step
# Epoch 61/100
# 851/851 - 2s - loss: 1877324702154752.0000 - 2s/epoch - 2ms/step
# Epoch 62/100
# 851/851 - 2s - loss: 1866660701011968.0000 - 2s/epoch - 2ms/step
# Epoch 63/100
# 851/851 - 1s - loss: 1856290368258048.0000 - 1s/epoch - 1ms/step
# Epoch 64/100
# 851/851 - 1s - loss: 1846044321120256.0000 - 1s/epoch - 1ms/step
# Epoch 65/100
# 851/851 - 1s - loss: 1836287229165568.0000 - 1s/epoch - 1ms/step
# Epoch 66/100
# 851/851 - 1s - loss: 1826649859424256.0000 - 1s/epoch - 1ms/step
# Epoch 67/100
# 851/851 - 1s - loss: 1817296091742208.0000 - 1s/epoch - 1ms/step
# Epoch 68/100
# 851/851 - 1s - loss: 1808202840670208.0000 - 1s/epoch - 1ms/step
# Epoch 69/100
# 851/851 - 1s - loss: 1799393460092928.0000 - 1s/epoch - 2ms/step
# Epoch 70/100
# 851/851 - 1s - loss: 1790945259421696.0000 - 1s/epoch - 1ms/step
# Epoch 71/100
# 851/851 - 2s - loss: 1782571549589504.0000 - 2s/epoch - 3ms/step
# Epoch 72/100
# 851/851 - 2s - loss: 1774556335308800.0000 - 2s/epoch - 2ms/step
# Epoch 73/100
# 851/851 - 1s - loss: 1766614135472128.0000 - 1s/epoch - 1ms/step
# Epoch 74/100
# 851/851 - 1s - loss: 1758577580572672.0000 - 1s/epoch - 1ms/step
# Epoch 75/100
# 851/851 - 1s - loss: 1750911063949312.0000 - 1s/epoch - 1ms/step
# Epoch 76/100
# 851/851 - 1s - loss: 1743559153680384.0000 - 1s/epoch - 1ms/step
# Epoch 77/100
# 851/851 - 2s - loss: 1736473128730624.0000 - 2s/epoch - 2ms/step
# Epoch 78/100
# 851/851 - 1s - loss: 1729367910645760.0000 - 1s/epoch - 1ms/step
# Epoch 79/100
# 851/851 - 1s - loss: 1722438383566848.0000 - 1s/epoch - 1ms/step
# Epoch 80/100
# 851/851 - 2s - loss: 1715619049242624.0000 - 2s/epoch - 2ms/step
# Epoch 81/100
# 851/851 - 2s - loss: 1709086135549952.0000 - 2s/epoch - 2ms/step
# Epoch 82/100
# 851/851 - 2s - loss: 1702544631922688.0000 - 2s/epoch - 2ms/step
# Epoch 83/100
# 851/851 - 1s - loss: 1696274113888256.0000 - 1s/epoch - 2ms/step
# Epoch 84/100
# 851/851 - 1s - loss: 1690244516675584.0000 - 1s/epoch - 1ms/step
# Epoch 85/100
# 851/851 - 1s - loss: 1684228072800256.0000 - 1s/epoch - 1ms/step
# Epoch 86/100
# 851/851 - 1s - loss: 1678272295337984.0000 - 1s/epoch - 1ms/step
# Epoch 87/100
# 851/851 - 1s - loss: 1672614883885056.0000 - 1s/epoch - 1ms/step
# Epoch 88/100
# 851/851 - 1s - loss: 1666959351480320.0000 - 1s/epoch - 1ms/step
# Epoch 89/100
# 851/851 - 1s - loss: 1661507293151232.0000 - 1s/epoch - 1ms/step
# Epoch 90/100
# 851/851 - 2s - loss: 1656100734631936.0000 - 2s/epoch - 2ms/step
# Epoch 91/100
# 851/851 - 2s - loss: 1650841681395712.0000 - 2s/epoch - 2ms/step
# Epoch 92/100
# 851/851 - 2s - loss: 1645834554834944.0000 - 2s/epoch - 2ms/step
# Epoch 93/100
# 851/851 - 1s - loss: 1640804745478144.0000 - 1s/epoch - 1ms/step
# Epoch 94/100
# 851/851 - 1s - loss: 1635917475348480.0000 - 1s/epoch - 1ms/step
# Epoch 95/100
# 851/851 - 1s - loss: 1631094092857344.0000 - 1s/epoch - 1ms/step
# Epoch 96/100
# 851/851 - 1s - loss: 1626393788022784.0000 - 1s/epoch - 1ms/step
# Epoch 97/100
# 851/851 - 1s - loss: 1621757907697664.0000 - 1s/epoch - 1ms/step
# Epoch 98/100
# 851/851 - 1s - loss: 1617382678200320.0000 - 1s/epoch - 1ms/step
# Epoch 99/100
# 851/851 - 1s - loss: 1612908261801984.0000 - 1s/epoch - 1ms/step
# Epoch 100/100
# 851/851 - 2s - loss: 1608664834113536.0000 - 2s/epoch - 2ms/step
# Mean Squared Error on Test Set: 1702906214481920.0
# 213/213 [==============================] - 1s 3ms/step

# Save Model